In [7]:
import os
import cv2
import random
import imgaug.augmenters as iaa
from tqdm.auto import tqdm

In [8]:
base_path = 'test'
# base_path = 'train'
desired_images = 300
# desired_images = 900

In [9]:
# Переберите все подпапки в основной папке
for folder_name in tqdm(os.listdir(base_path)):
    folder_path = os.path.join(base_path, folder_name)
    
    # Если текущий элемент является папкой
    if os.path.isdir(folder_path):
        
        # Подсчитайте количество изображений в текущей папке
        num_images = len(os.listdir(folder_path))
        
        # Если количество изображений меньше желаемого количества
        if num_images < desired_images:
            
            # Создайте аугментер
            seq = iaa.Sequential([
                iaa.Fliplr(0.5), # Отзеркалить изображение слева направо
                iaa.Affine(rotate=(-30, 30)), # Повернуть изображение на случайный угол 
                # Small gaussian blur with random sigma between 0 and 0.5.
                # But we only blur about 50% of all images.
                iaa.Sometimes(0.1,iaa.GaussianBlur(sigma=(0, 3.0))),
                # Strengthen or weaken the contrast in each image.
                iaa.LinearContrast((0.75, 1.5)),
                iaa.Crop(px=(0, 16)), # random crops
                # Add gaussian noise.
                # For 50% of all images, we sample the noise once per pixel.
                # For the other 50% of all images, we sample the noise per pixel AND
                # channel. This can change the color (not only brightness) of the
                # pixels.
                iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
                # Apply affine transformations to each image.
                # Scale/zoom them, translate/move them, rotate them and shear them.
                iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                           translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                           rotate=(-25, 25),
                           shear=(-8, 8)
                           )
            ])
            
            # Переберите недостающее количество изображений
            for i in tqdm(range(desired_images - num_images)):
                
                # Выберите случайное изображение из текущей папки
                image_name = random.choice(os.listdir(folder_path))
                image_path = os.path.join(folder_path, image_name)
                
                # Проверьте, что изображение прочитано корректно
                image = cv2.imread(image_path)
                if image is None:
                    print(f"Could not read image: {image_path}")
                    continue
                
                # Примените к изображению аугментацию
                image_aug = seq(image=image)
                
                # Создайте новое имя файла
                new_image_name = f"{i+num_images+1}.jpg"
                new_image_path = os.path.join(folder_path, new_image_name)
                
                # Сохраните аугментированное изображение в текущей папке
                cv2.imwrite(new_image_path, image_aug)

 12%|█▏        | 4/34 [00:12<01:23,  2.78s/it]

Could not read image: test\bulb\bulb (108).jpg


Could not read image: test\bulb\bulb (108).jpg


100%|██████████| 34/34 [02:02<00:00,  3.60s/it]
